In [24]:
!pip install dwave-ocean-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 21.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.9/158.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.1/605.1 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.4/893.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.7/702.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.9/952.9 kB 23.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

## **Imports**

In [3]:
import numpy as np
import itertools
import os
import autograd.numpy as np2
import pennylane as qml


from dimod import BQM
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from dimod.reference.samplers import ExactSolver
from dwave.system import DWaveSampler, EmbeddingComposite
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv('DWA_API_KEY')

## **1. From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)**

### **Define the ILP formulation of the BPP**

In [27]:

def BPP_ILP_Program(object_sizes, container_capacity):
    num_objects = len(object_sizes)  # Number of objects
    max_containers = num_objects  # Maximum number of containers (can have up to num_objects containers)

        # Create the quadratic program
    qp = QuadraticProgram()

    # Binary variables for containers
    container_usage_variables = {}
    for container in range(max_containers):
        container_var = f"y{container + 1}"  # Variable to indicate if the container is in use
        container_usage_variables[container_var] = 1  # Coefficient for minimization
        qp.binary_var(container_var)
    
    # Objective function: minimize the number of containers used
    qp.minimize(linear=container_usage_variables)

        # Constraint 1: Each object must be assigned to exactly one container
    for obj in range(num_objects):
        object_assignment_vars = {}
        for container in range(max_containers):
            assignment_var = f"x{obj+1}_{container+1}"  # Variable for object assignment to containers
            object_assignment_vars[assignment_var] = 1
            qp.binary_var(assignment_var)
        qp.linear_constraint(linear=object_assignment_vars, sense="==", rhs=1, name=f"x_{obj + 1}_assigned_const")

    # Constraint 2: The total size of objects in each container must not exceed its capacity
    for container in range(max_containers):
        container_load_vars = {}
        for obj in range(num_objects):
            assignment_var = f"x{obj+1}_{container+1}"
            container_load_vars[assignment_var] = object_sizes[obj]  # Size of the object
        
        container_var = f"y{container + 1}"
        container_load_vars[container_var] = -container_capacity  # Container capacity
        qp.linear_constraint(linear=container_load_vars, sense="<=", rhs=0, name=f"y_{container+1}_capacity_const")

    return qp

def quadratic_program_to_qubo(qp):
    conv = QuadraticProgramToQubo()
    qubo_problem = conv.convert(qp)
    return qubo_problem


# Problem parameters
object_sizes = [4, 8, 1, 4]  # Sizes of the objects
container_capacity = 10  # Capacity of the containers

qp = BPP_ILP_Program(object_sizes, container_capacity)



### **Create a function to transform the ILP model into a QUBO**

In [28]:
qubo_problem = quadratic_program_to_qubo(qp)

# Print the problem formulation
print(qubo_problem.prettyprint())

Problem name: 

Minimize
  85*x1_1^2 + 10*x1_1*x1_2 + 10*x1_1*x1_3 + 10*x1_1*x1_4 + 320*x1_1*x2_1
  + 40*x1_1*x3_1 + 160*x1_1*x4_1 + 40*x1_1*y_1_capacity_const@int_slack@0
  + 80*x1_1*y_1_capacity_const@int_slack@1
  + 160*x1_1*y_1_capacity_const@int_slack@2
  + 120*x1_1*y_1_capacity_const@int_slack@3 + 85*x1_2^2 + 10*x1_2*x1_3
  + 10*x1_2*x1_4 + 320*x1_2*x2_2 + 40*x1_2*x3_2 + 160*x1_2*x4_2
  + 40*x1_2*y_2_capacity_const@int_slack@0
  + 80*x1_2*y_2_capacity_const@int_slack@1
  + 160*x1_2*y_2_capacity_const@int_slack@2
  + 120*x1_2*y_2_capacity_const@int_slack@3 + 85*x1_3^2 + 10*x1_3*x1_4
  + 320*x1_3*x2_3 + 40*x1_3*x3_3 + 160*x1_3*x4_3
  + 40*x1_3*y_3_capacity_const@int_slack@0
  + 80*x1_3*y_3_capacity_const@int_slack@1
  + 160*x1_3*y_3_capacity_const@int_slack@2
  + 120*x1_3*y_3_capacity_const@int_slack@3 + 85*x1_4^2 + 320*x1_4*x2_4
  + 40*x1_4*x3_4 + 160*x1_4*x4_4 + 40*x1_4*y_4_capacity_const@int_slack@0
  + 80*x1_4*y_4_capacity_const@int_slack@1
  + 160*x1_4*y_4_capacity_const@int_s

### **Example BPP small**

In [29]:
sizes = [3, 2, 5]
bin_capacity = 6
model_ilp_small = BPP_ILP_Program(sizes, bin_capacity)
print("ILP VERSION",model_ilp_small.prettyprint())

qubo_problem_small = quadratic_program_to_qubo(model_ilp_small)

print("QUBO VERSION",qubo_problem_small.prettyprint())


ILP VERSION Problem name: 

Minimize
  y1 + y2 + y3

Subject to
  Linear constraints (6)
    x1_1 + x1_2 + x1_3 == 1  'x_1_assigned_const'
    x2_1 + x2_2 + x2_3 == 1  'x_2_assigned_const'
    x3_1 + x3_2 + x3_3 == 1  'x_3_assigned_const'
    3*x1_1 + 2*x2_1 + 5*x3_1 - 6*y1 <= 0  'y_1_capacity_const'
    3*x1_2 + 2*x2_2 + 5*x3_2 - 6*y2 <= 0  'y_2_capacity_const'
    3*x1_3 + 2*x2_3 + 5*x3_3 - 6*y3 <= 0  'y_3_capacity_const'

  Binary variables (12)
    y1 y2 y3 x1_1 x1_2 x1_3 x2_1 x2_2 x2_3 x3_1 x3_2 x3_3

QUBO VERSION Problem name: 

Minimize
  40*x1_1^2 + 8*x1_1*x1_2 + 8*x1_1*x1_3 + 48*x1_1*x2_1 + 120*x1_1*x3_1
  + 24*x1_1*y_1_capacity_const@int_slack@0
  + 48*x1_1*y_1_capacity_const@int_slack@1
  + 72*x1_1*y_1_capacity_const@int_slack@2 + 40*x1_2^2 + 8*x1_2*x1_3
  + 48*x1_2*x2_2 + 120*x1_2*x3_2 + 24*x1_2*y_2_capacity_const@int_slack@0
  + 48*x1_2*y_2_capacity_const@int_slack@1
  + 72*x1_2*y_2_capacity_const@int_slack@2 + 40*x1_3^2 + 48*x1_3*x2_3
  + 120*x1_3*x3_3 + 24*x1_3*y_3_capac

### **Example BPP medium**

In [8]:
sizes = [3, 2, 5, 4, 7, 6]
bin_capacity = 10

model_ilp_medium = BPP_ILP_Program(sizes, bin_capacity)
print("ILP VERSION",model_ilp_medium.prettyprint())

qubo_problem_medium = quadratic_program_to_qubo(model_ilp_medium)

print("QUBO VERSION",qubo_problem.prettyprint())


ILP VERSION Problem name: 

Minimize
  y1 + y2 + y3 + y4 + y5 + y6

Subject to
  Linear constraints (12)
    x1_1 + x1_2 + x1_3 + x1_4 + x1_5 + x1_6 == 1  'x_1_assigned_const'
    x2_1 + x2_2 + x2_3 + x2_4 + x2_5 + x2_6 == 1  'x_2_assigned_const'
    x3_1 + x3_2 + x3_3 + x3_4 + x3_5 + x3_6 == 1  'x_3_assigned_const'
    x4_1 + x4_2 + x4_3 + x4_4 + x4_5 + x4_6 == 1  'x_4_assigned_const'
    x5_1 + x5_2 + x5_3 + x5_4 + x5_5 + x5_6 == 1  'x_5_assigned_const'
    x6_1 + x6_2 + x6_3 + x6_4 + x6_5 + x6_6 == 1  'x_6_assigned_const'
    3*x1_1 + 2*x2_1 + 5*x3_1 + 4*x4_1 + 7*x5_1 + 6*x6_1 - 10*y1
    <= 0  'y_1_capacity_const'
    3*x1_2 + 2*x2_2 + 5*x3_2 + 4*x4_2 + 7*x5_2 + 6*x6_2 - 10*y2
    <= 0  'y_2_capacity_const'
    3*x1_3 + 2*x2_3 + 5*x3_3 + 4*x4_3 + 7*x5_3 + 6*x6_3 - 10*y3
    <= 0  'y_3_capacity_const'
    3*x1_4 + 2*x2_4 + 5*x3_4 + 4*x4_4 + 7*x5_4 + 6*x6_4 - 10*y4
    <= 0  'y_4_capacity_const'
    3*x1_5 + 2*x2_5 + 5*x3_5 + 4*x4_5 + 7*x5_5 + 6*x6_5 - 10*y5
    <= 0  'y_5_capacity_c

### **Example BPP large**

In [9]:
sizes = [3, 2, 5, 4, 7, 6, 1, 8, 9, 10]
bin_capacity = 15

model_ilp_large = BPP_ILP_Program(sizes, bin_capacity)
print("ILP VERSION",model_ilp_large.prettyprint())

qubo_problem_large = quadratic_program_to_qubo(model_ilp_large)

print("QUBO VERSION",qubo_problem.prettyprint())


ILP VERSION Problem name: 

Minimize
  y1 + y10 + y2 + y3 + y4 + y5 + y6 + y7 + y8 + y9

Subject to
  Linear constraints (20)
    x1_1 + x1_10 + x1_2 + x1_3 + x1_4 + x1_5 + x1_6 + x1_7 + x1_8 + x1_9
    == 1  'x_1_assigned_const'
    x2_1 + x2_10 + x2_2 + x2_3 + x2_4 + x2_5 + x2_6 + x2_7 + x2_8 + x2_9
    == 1  'x_2_assigned_const'
    x3_1 + x3_10 + x3_2 + x3_3 + x3_4 + x3_5 + x3_6 + x3_7 + x3_8 + x3_9
    == 1  'x_3_assigned_const'
    x4_1 + x4_10 + x4_2 + x4_3 + x4_4 + x4_5 + x4_6 + x4_7 + x4_8 + x4_9
    == 1  'x_4_assigned_const'
    x5_1 + x5_10 + x5_2 + x5_3 + x5_4 + x5_5 + x5_6 + x5_7 + x5_8 + x5_9
    == 1  'x_5_assigned_const'
    x6_1 + x6_10 + x6_2 + x6_3 + x6_4 + x6_5 + x6_6 + x6_7 + x6_8 + x6_9
    == 1  'x_6_assigned_const'
    x7_1 + x7_10 + x7_2 + x7_3 + x7_4 + x7_5 + x7_6 + x7_7 + x7_8 + x7_9
    == 1  'x_7_assigned_const'
    x8_1 + x8_10 + x8_2 + x8_3 + x8_4 + x8_5 + x8_6 + x8_7 + x8_8 + x8_9
    == 1  'x_8_assigned_const'
    x9_1 + x9_10 + x9_2 + x9_3 + x9_4 + x9

## **2. Create a Brute Force solver for the QUBO problem and solve the specific instances.**

In [1]:
def defined_problem_using_dmod(object_sizes, container_capacity):
    num_objects = len(object_sizes)  # Number of objects
    max_containers = num_objects  # Maximum number of containers (can have up to num_objects containers)

    # Create a binary quadratic model (BQM)
    bqm = BQM("BINARY")
    l1 = 5  # Lagrange multiplier

    # Add linear terms for containers (minimize the number of containers used)
    for container in range(max_containers):
        container_var = f"y_{container}"  # Variable to indicate if the container is in use
        bqm.add_linear(container_var, 1)

    # Add constraints: each object must be placed in exactly one container
    for obj in range(num_objects):
        constraint_one = [(f"x_{obj}_{container}", 1) for container in range(max_containers)]  # Coefficients list
        bqm.add_linear_equality_constraint(constraint_one, constant=-1, lagrange_multiplier=l1)

    # Add capacity constraints: total size of objects in a container should not exceed the container's capacity
    for container in range(max_containers):
        constraint_two = [(f"x_{obj}_{container}", object_sizes[obj]) for obj in range(num_objects)]
        constraint_two.append((f"y_{container}", -container_capacity))
        bqm.add_linear_inequality_constraint(constraint_two, label=f"y_{container} constraint", ub=0, lagrange_multiplier=l1)

    return bqm

def get_QUBO_matrix(bqm):
    # Convert BQM to QUBO representation
    qubo, offset = bqm.to_qubo()

    # Get all variables from QUBO
    variables = list(bqm.variables)

    # Create a mapping of variables to indices
    variable_index = {var: idx for idx, var in enumerate(variables)}

    # Initialize the QUBO matrix with zeros
    n = len(variables)
    qubo_matrix = np.zeros((n, n))

    # Fill in the QUBO matrix using the terms from the QUBO representation
    for (var1, var2), bias in qubo.items():
        i, j = variable_index[var1], variable_index[var2]
        qubo_matrix[i, j] = bias

    return qubo_matrix, offset, variables

def solve_qubo_brute_force(Q):
    # Number of variables (from the QUBO matrix size)
    n = Q.shape[0]
    
    # Generate all binary combinations of length n
    possible_solutions = list(itertools.product([0, 1], repeat=n))
    
    # Initialize variables to store the best solution and minimum value
    best_solution = None
    min_value = float('inf')
    
    # Loop through all possible binary vectors x
    for x in possible_solutions:
        # Convert x to a column vector
        x_vec = np.array(x).reshape(-1, 1)
        
        # Calculate the objective function value: x^T Q x
        value = np.dot(np.dot(x_vec.T, Q), x_vec)[0, 0]
        
        # If this is the best value so far, store it
        if value < min_value:
            min_value = value
            best_solution = x
    
    # Return the best solution and its objective function value
    return best_solution, min_value

def summarize_solution(solution, object_sizes, container_capacity):
    used_containers = []
    unused_containers = []
    object_placement = {}

    # Iterate through the solution to find container and object placements
    for var, value in solution:
        if var.startswith('y_'):
            container_index = int(var.split('_')[1])
            if value == 1:
                used_containers.append(container_index)
            else:
                unused_containers.append(container_index)
        
        elif var.startswith('x_') and value == 1:
            # Object is placed in a container
            obj_index, container_index = map(int, var.split('_')[1:])
            if container_index not in object_placement:
                object_placement[container_index] = []
            object_placement[container_index].append(obj_index)

    # Sort the containers for better presentation
    used_containers.sort()
    unused_containers.sort()

    # Print a summary of the solution
    print(f"Container capacity: {container_capacity}")
    print(f"Object sizes: {object_sizes}")
    print(f"Number of containers used: {len(used_containers)}")
    print(f"Containers used: {used_containers}")
    
    if unused_containers:
        print(f"Containers not used: {unused_containers}")
    else:
        print("All containers were used.")
    
    print("\nObject placements and load:")
    for container in used_containers:
        objects_in_container = object_placement.get(container, [])
        total_weight = sum(object_sizes[obj] for obj in objects_in_container)
        print(f"  Container {container}: Objects {objects_in_container}, Total weight: {total_weight}/{container_capacity}")
    
    print("\nSummary:")
    total_objects = sum(len(obj_list) for obj_list in object_placement.values())
    print(f"Total number of objects: {total_objects}")
    print(f"Total weight distributed: {sum(object_sizes)}")

### **Solution of example using brute force (small)**

In [13]:
# Problem parameters
sizes_small = [4, 3, 1]  # Sizes of the objects
bin_capacity_small = 5  # Capacity of the containers

# Generate the BQM for the problem
bqm_small = defined_problem_using_dmod(sizes_small, bin_capacity_small)

# Create the QUBO matrix
qubo_matrix_small, offset_small , variables_small = get_QUBO_matrix(bqm_small)

# Solve the QUBO matrix
best_solution_small, min_value_small = solve_qubo_brute_force(qubo_matrix_small)

solution = [ (var,value) for var,value in zip(variables_small, best_solution_small)]


print("Vector solutions:", solution)
print("Min value:", min_value_small)
summarize_solution(solution, sizes, bin_capacity)

Vector solutions: [('y_0', 0), ('y_1', 1), ('y_2', 1), ('x_0_0', 0), ('x_0_1', 0), ('x_0_2', 1), ('x_1_0', 0), ('x_1_1', 1), ('x_1_2', 0), ('x_2_0', 0), ('x_2_1', 0), ('x_2_2', 1), ('slack_y_0 constraint_0', 0), ('slack_y_0 constraint_1', 0), ('slack_y_0 constraint_2', 0), ('slack_y_1 constraint_0', 0), ('slack_y_1 constraint_1', 0), ('slack_y_1 constraint_2', 1), ('slack_y_2 constraint_0', 0), ('slack_y_2 constraint_1', 0), ('slack_y_2 constraint_2', 0)]
Min value: -13.0
Container capacity: 5
Object sizes: [4, 3, 1]
Number of containers used: 2
Containers used: [1, 2]
Containers not used: [0]

Object placements and load:
  Container 1: Objects [1], Total weight: 3/5
  Container 2: Objects [0, 2], Total weight: 5/5

Summary:
Total number of objects: 3
Total weight distributed: 8


### **Solution of example using brute force (medium)**

If medium failed trying large wouldn't be feasible

In [18]:
sizes_med = [3, 2, 5, 4, 7, 6]
bin_capacity_med = 10

# Generate the BQM for the problem
bqm_med = defined_problem_using_dmod(sizes_med, bin_capacity_med)

In [17]:
# Create the QUBO matrix
qubo_matrix_med, offset_med , variables_med = get_QUBO_matrix(bqm_med)

# Solve the QUBO matrix
best_solution_med, min_value_med = solve_qubo_brute_force(qubo_matrix_med)

solution = [ (var,value) for var,value in zip(variables_med, best_solution_med)]


print("Vector solutions:", solution)
print("Min value:", min_value_med)
summarize_solution(solution, sizes, bin_capacity)


: 

## **3. To solve the QUBO, use quantum annealing simulators.**

In [11]:
def solve_qubo_by_annealing(bqm, reads = 1000):
    sampler = EmbeddingComposite(DWaveSampler(token = api_key))
    Q, offset = bqm.to_qubo()
    # Solve the QUBO problem using quantum annealing
    sampleset = sampler.sample_qubo(Q, num_reads=reads) 
    return sampleset

## **Annealing small**

In [14]:
sampleset = solve_qubo_by_annealing(bqm_small, 1000)
best_solution = sampleset.first.sample
min_value = sampleset.first.energy

solution = list(best_solution.items())
print("Vector solutions:", solution)
print("Min value:", min_value)

summarize_solution(solution, sizes_small, bin_capacity_small)

Vector solutions: [('slack_y_0 constraint_0', 0), ('slack_y_0 constraint_1', 0), ('slack_y_0 constraint_2', 0), ('slack_y_1 constraint_0', 1), ('slack_y_1 constraint_1', 0), ('slack_y_1 constraint_2', 0), ('slack_y_2 constraint_0', 1), ('slack_y_2 constraint_1', 0), ('slack_y_2 constraint_2', 0), ('x_0_0', 0), ('x_0_1', 0), ('x_0_2', 1), ('x_1_0', 0), ('x_1_1', 1), ('x_1_2', 0), ('x_2_0', 0), ('x_2_1', 1), ('x_2_2', 0), ('y_0', 0), ('y_1', 1), ('y_2', 1)]
Min value: -13.0
Container capacity: 5
Object sizes: [4, 3, 1]
Number of containers used: 2
Containers used: [1, 2]
Containers not used: [0]

Object placements and load:
  Container 1: Objects [1, 2], Total weight: 4/5
  Container 2: Objects [0], Total weight: 4/5

Summary:
Total number of objects: 3
Total weight distributed: 8


## **Annealing medium**

In [20]:
sampleset = solve_qubo_by_annealing(bqm_med, 2500)
best_solution = sampleset.first.sample
min_value = sampleset.first.energy

solution = list(best_solution.items())
print("Vector solutions:", solution)
print("Min value:", min_value)

summarize_solution(solution, sizes_med, bin_capacity_med)

Vector solutions: [('slack_y_0 constraint_0', 0), ('slack_y_0 constraint_1', 1), ('slack_y_0 constraint_2', 0), ('slack_y_0 constraint_3', 1), ('slack_y_1 constraint_0', 1), ('slack_y_1 constraint_1', 0), ('slack_y_1 constraint_2', 0), ('slack_y_1 constraint_3', 1), ('slack_y_2 constraint_0', 0), ('slack_y_2 constraint_1', 0), ('slack_y_2 constraint_2', 1), ('slack_y_2 constraint_3', 0), ('slack_y_3 constraint_0', 1), ('slack_y_3 constraint_1', 1), ('slack_y_3 constraint_2', 1), ('slack_y_3 constraint_3', 0), ('slack_y_4 constraint_0', 0), ('slack_y_4 constraint_1', 0), ('slack_y_4 constraint_2', 0), ('slack_y_4 constraint_3', 0), ('slack_y_5 constraint_0', 1), ('slack_y_5 constraint_1', 1), ('slack_y_5 constraint_2', 0), ('slack_y_5 constraint_3', 1), ('x_0_0', 0), ('x_0_1', 0), ('x_0_2', 0), ('x_0_3', 1), ('x_0_4', 0), ('x_0_5', 0), ('x_1_0', 0), ('x_1_1', 1), ('x_1_2', 0), ('x_1_3', 0), ('x_1_4', 0), ('x_1_5', 0), ('x_2_0', 0), ('x_2_1', 0), ('x_2_2', 0), ('x_2_3', 0), ('x_2_4', 0),

In [24]:
sizes_large = [3, 2, 5, 4, 7, 6, 1, 8, 9, 10]
bin_capacity_large = 15

# Generate the BQM for the problem
bqm_large = defined_problem_using_dmod(sizes_large, bin_capacity_large)

sampleset = solve_qubo_by_annealing(bqm_large, 3000)
best_solution = sampleset.first.sample
min_value = sampleset.first.energy

solution = list(best_solution.items())
print("Vector solutions:", solution)
print("Min value:", min_value)

summarize_solution(solution, sizes_large, bin_capacity_large)

Vector solutions: [('slack_y_0 constraint_0', 0), ('slack_y_0 constraint_1', 1), ('slack_y_0 constraint_2', 0), ('slack_y_0 constraint_3', 0), ('slack_y_1 constraint_0', 0), ('slack_y_1 constraint_1', 0), ('slack_y_1 constraint_2', 0), ('slack_y_1 constraint_3', 0), ('slack_y_2 constraint_0', 0), ('slack_y_2 constraint_1', 0), ('slack_y_2 constraint_2', 0), ('slack_y_2 constraint_3', 0), ('slack_y_3 constraint_0', 1), ('slack_y_3 constraint_1', 1), ('slack_y_3 constraint_2', 0), ('slack_y_3 constraint_3', 0), ('slack_y_4 constraint_0', 0), ('slack_y_4 constraint_1', 0), ('slack_y_4 constraint_2', 0), ('slack_y_4 constraint_3', 0), ('slack_y_5 constraint_0', 1), ('slack_y_5 constraint_1', 0), ('slack_y_5 constraint_2', 0), ('slack_y_5 constraint_3', 1), ('slack_y_6 constraint_0', 0), ('slack_y_6 constraint_1', 1), ('slack_y_6 constraint_2', 1), ('slack_y_6 constraint_3', 0), ('slack_y_7 constraint_0', 0), ('slack_y_7 constraint_1', 0), ('slack_y_7 constraint_2', 0), ('slack_y_7 constrai

## **4. Use a Quantum Variational approach to solve the QUBO.**

In [21]:
Q = qubo_matrix_small
n_wires = Q.shape[0]
dev = qml.device('default.qubit', wires=n_wires)

def run_an_experiment(optimizer, weights, steps=100, rotation = qml.RX):

    @qml.qnode(dev)
    def circuit(weights):
        qml.BasicEntanglerLayers(weights=weights, wires=range(n_wires), rotation=rotation)
        return qml.numpy.array([qml.expval(qml.Z(i)) for i in range(n_wires)])
    
    def qubo_cost_function(expectation):
        binary_solution = ( np2.tanh(expectation) + 1 ) / 2
        return np.dot(binary_solution.T, np.dot(Q, binary_solution))

    def cost(weights):
        expectation = circuit(weights) 
        energy = qubo_cost_function(expectation)
        return energy


    params = weights.copy()

    for step in range(steps):
        params, energy = optimizer.step_and_cost(cost, params)
        final_expectation = circuit(params)
        qubo_solution = ( np.sign(final_expectation) + 1 ) / 2
        print(f"STEP {step + 1}: {energy}, {qubo_solution}")

    return np.array(qubo_solution), energy, circuit



shape = qml.BasicEntanglerLayers.shape(n_layers=3, n_wires=n_wires)
weights = qml.numpy.random.random(size=shape, requires_grad=True)

# Configurar el optimizador
optimizer = qml.AdamOptimizer(0.1)
# optimizer = qml.NesterovMomentumOptimizer(0.5)

## **Running experiment with RX rotation circuit**

In [12]:
qubo_solution, energy, circuit = run_an_experiment(optimizer, weights, 100)

STEP 1: 253.75907432373765, [1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1.]
STEP 2: 251.89262567812165, [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.]
STEP 3: 251.71361127833754, [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1.]
STEP 4: 251.75850206862015, [1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1.]
STEP 5: 251.75506586046401, [1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1.]
STEP 6: 251.6871381819785, [1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1.]
STEP 7: 251.55811730001648, [1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1.]
STEP 8: 251.36253512669936, [1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1.]
STEP 9: 251.0798819420952, [1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1.]
STEP 10: 250.67078553830748, [1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1.]
STEP 11: 250.07603953066385, [1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.

(tensor([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.], requires_grad=True),
 11.86854807160335)

## **Running experiment with RY rotation circuit**

In [17]:
qubo_solution, energy, circuit = run_an_experiment(optimizer, weights, 150, qml.RY)

STEP 1: 270.1451722738467, [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
STEP 2: 254.5429793671955, [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
STEP 3: 246.06605355106453, [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
STEP 4: 241.68871006394997, [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
STEP 5: 238.1805449230455, [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0.]
STEP 6: 233.18004981787158, [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
STEP 7: 225.88686065290787, [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
STEP 8: 216.38210661702217, [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
STEP 9: 204.90962538982745, [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
STEP 10: 190.68407173672065, [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1.]


KeyboardInterrupt: 

## **5. Use QAOA to solve the QUBO USING PENNYLANE (FAILED)**

In [42]:
import pennylane as qml
import numpy as np

from collections import defaultdict


# Define QUBO matrix
Q = qubo_matrix_small
num_qubits = len(Q)
p = 3  # Number of QAOA layers


def from_Q_to_Ising(Q, offset):
    """Convert the matrix Q of Eq.3 into Eq.13 elements J and h"""
    n_qubits = len(Q)  # Get the number of qubits (variables) in the QUBO matrix
    # Create default dictionaries to store h and pairwise interactions J
    h = defaultdict(int)
    J = defaultdict(int)

    # Loop over each qubit (variable) in the QUBO matrix
    for i in range(n_qubits):
        # Update the magnetic field for qubit i based on its diagonal element in Q
        h[(i,)] -= Q[i, i] / 2
        # Update the offset based on the diagonal element in Q
        offset += Q[i, i] / 2
        # Loop over other qubits (variables) to calculate pairwise interactions
        for j in range(i + 1, n_qubits):
            # Update the pairwise interaction strength (J) between qubits i and j
            J[(i, j)] += Q[i, j] / 4
            # Update the magnetic fields for qubits i and j based on their interactions in Q
            h[(i,)] -= Q[i, j] / 4
            h[(j,)] -= Q[i, j] / 4
            # Update the offset based on the interaction strength between qubits i and j
            offset += Q[i, j] / 4
    # Return the magnetic fields, pairwise interactions, and the updated offset
    return h, J, offset

h, J, offset = from_Q_to_Ising(Q, 0)

# Define the device
dev = qml.device("default.qubit", wires=num_qubits)

# Define the QAOA circuit
def qaoa_layer(gamma, beta):
    # Apply phase separation based on Ising Hamiltonian
    for i in range(num_qubits):
        qml.RZ(-2 * gamma * h[i], wires=i)
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            if J[i, j] != 0:
                qml.CNOT(wires=[i, j])
                qml.RZ(-2 * gamma * J[i, j], wires=j)
                qml.CNOT(wires=[i, j])
                
    # Apply mixing Hamiltonian
    for i in range(num_qubits):
        qml.RX(2 * beta, wires=i)

@qml.qnode(dev)
def qaoa_circuit(params):
    # Apply the QAOA ansatz with p layers
    gammas = params[:p]
    betas = params[p:]
    for i in range(p):
        qaoa_layer(gammas[i], betas[i])
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

# Define cost function based on QUBO matrix and expectation values
def energy_cost_function(expectation):
    binary_solution = (np2.tanh(qml.numpy.array(expectation)) + 1) / 2
    return np.dot(binary_solution, Q @ binary_solution)

# Full QAOA cost function
def qaoa_cost(params):
    expectation = qaoa_circuit(params)
    return energy_cost_function(expectation, h, J,)

# Initialize parameters and set up the optimizer
initial_params = qml.numpy.random.rand(2 * p, requires_grad=True) * np.pi  # Random initial angles
opt = qml.AdamOptimizer(stepsize=0.15)
num_steps = 100

# Optimization loop
params = initial_params
for step in range(num_steps):
    params, cost = opt.step_and_cost(qaoa_cost, params)
    if step % 1 == 0:
        final_expectation = qaoa_circuit(params)
        qubo_solution = ( np.sign(final_expectation) + 1 ) / 2
        print(f"Step {step} - Cost: {cost} - Solution: {qubo_solution}, {final_expectation}")

# Optimized parameters
# print("Optimized QAOA parameters:", params)


IndexError: list index out of range

## **5. Use QAOA to solve the QUBO USING QUISKIT**

In [31]:
# Import necessary classes from Qiskit libraries
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import ADAM,COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import random
import numpy as np



bpp_problem = qubo_problem_small
# bpp_problem = qubo_problem_medium
# bpp_problem = qubo_problem_large

steps = 5
repetitions_layers = 2
initial_parameters = [ random.random()  for _ in range(2 * repetitions_layers)]

# Set up the QAOA algorithm
general_sampler = Sampler()   # Sampler to evaluate the quantum circuit
optimizer_1 = ADAM(maxiter=steps)  
optimizer_2 = COBYLA(maxiter=steps)  



def solve_qubo_by_qaoa(sampler, optimizer, repetitions_layers, initial_parameters, problem):
    # Create QAOA instance with the sampler, optimizer, and initial parameters
    qaoa = QAOA(sampler = sampler, optimizer = optimizer, reps = repetitions_layers, initial_point = initial_parameters)

    # Set up the optimization problem for QAOA
    optimization_problem_qaoa = MinimumEigenOptimizer(qaoa)

    # Solve the problem and retrieve the result
    solution = optimization_problem_qaoa.solve(problem)

    return solution


solution = solve_qubo_by_qaoa(general_sampler, optimizer_1, repetitions_layers, initial_parameters, bpp_problem)

/var/folders/lw/fs8gsplj1d9fn88nj8c030q80000gn/T/ipykernel_24169/2941631624.py:20: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  general_sampler = Sampler()   # Sampler to evaluate the quantum circuit


In [32]:
solution

<MinimumEigenOptimizationResult: fval=2.0, y1=1.0, y2=1.0, y3=0.0, x1_1=0.0, x1_2=1.0, x1_3=0.0, x2_1=0.0, x2_2=1.0, x2_3=0.0, x3_1=1.0, x3_2=0.0, x3_3=0.0, y_1_capacity_const@int_slack@0=1.0, y_1_capacity_const@int_slack@1=0.0, y_1_capacity_const@int_slack@2=0.0, y_2_capacity_const@int_slack@0=1.0, y_2_capacity_const@int_slack@1=0.0, y_2_capacity_const@int_slack@2=0.0, y_3_capacity_const@int_slack@0=0.0, y_3_capacity_const@int_slack@1=0.0, y_3_capacity_const@int_slack@2=0.0, status=SUCCESS>